In [ ]:
%load_ext autoreload
%autoreload 2
from awesome.run.awesome_config import AwesomeConfig
from awesome.run.awesome_runner import AwesomeRunner
from awesome.util.reflection import class_name
from awesome.analytics.result_model import ResultModel
from awesome.util.path_tools import get_project_root_path, get_package_root_path
import os
import torch
import re
from awesome.util.format import latex_postprocessor

os.chdir(get_project_root_path()) # Beeing in the root directory of the project is important for the relative paths to work consistently

In [ ]:
from awesome.analytics.result_comparison import ResultComparison

paths = ['./runs/unireps/joint','./runs/unireps/individual' ]
models = []

for path in paths:
    for folder in os.listdir(path):
        model = ResultModel.from_path(os.path.join(path, folder))
        models.append(model)

import re
p = r"#?(?P<cfg_num>\d+)?_?(?P<net>\w+)_benchmark(?P<feat>(\+\w+)*)\_(?P<date>\d{2}_\d{2}_\d{2})\_(?P<time>\d{2}_\d{2}_\d{2})"
pattern = re.compile(p)

for model in models:
    match = pattern.match(model.name)
    model_name = None
    feat = []
    if match:
        model_name = match.group('net')
        features = match.group('feat')
        if features is not None and features != "":
            feat = features.strip("+").split("+")
            if not any(["seed" in x for x in feat]):
                feat.append("seed42")
            feat = sorted(feat)
    else:
        print('No match for', model.name)
    model_name = model_name.replace("NET", "Net")
    model.display_name = model_name + " " + " ".join(feat)
    model.config.result_directory = "final_mask"
    model.save_config()


# Resort the models by name to get a meaningful table order

_order = []

models = sorted(models, key=lambda m: _order.index(m.name) if m.name in _order else 0)

comparison = ResultComparison(models)
comparison.assign_numbers(force=True)

os.environ['PLOT_OUTPUT_DIR'] = comparison.output_folder

save_args = dict(transparent=False, save=True, dpi=300, ext=["png", "pdf"])

models

In [ ]:
from typing import Any, Dict
import pandas as pd
model_names = set([m.config.name.split(" ")[0] for m in models])
features = set([y for m in models for y in m.config.name.split(" ")[1:]])
seeds = {x for x in features if "seed" in x}
features = features - seeds - {"convex", "joint"}
model_names, features, seeds

columns = list(features) + ['joint ' + x for x in features]
columns = columns + [x + " seeds" for x in columns]

rows = list(model_names)

df = pd.DataFrame(index=rows, columns=columns)
df[pd.isna(df)] = 0

for c in [x for x in df.columns if "seeds" in x]:
    df.loc[:, c] = ""

def extract_features(model: ResultModel) -> Dict[str, Any]:
    res = dict()
    
    res['joint'] = "joint" in model.config.name
    
    res['model_name'] = model.config.name.split(" ")[0]
    model_features = model.config.name.split(" ")[1:]
    
    if res['joint']:
        model_features.remove("joint")
    
    seed = [x for x in model_features if "seed" in x][0]
    model_features.remove(seed)

    res['seed'] = int(seed.replace("seed", ""))

    if "convex" in model_features:
        model_features.remove("convex")
        res['convex'] = True
    else:
        res['convex'] = False

    assert len(model_features) == 1, f"Multiple features {model_features} in model {model.output_folder}"
    res['feature_type'] = model_features[0]

    return res


for model in models:
    res = extract_features(model)

    is_joint = res['joint']
    model_name =  res['model_name']
    model_features = model.config.name.split(" ")[1:]
    
    feat_type = "joint " + res['feature_type'] if is_joint else res['feature_type']   
    df.loc[model_name, feat_type] += 1
    seed_col = feat_type + " seeds"
    df.loc[model_name, seed_col] = str(df.loc[model_name, seed_col]) + " " + str(res['seed'])
    

df

In [ ]:
from typing import Tuple

metrics = [
    "eval/epoch/MeanForegroundBinaryMIOU" ,
    "eval/epoch/MeanPixelAccuracy",
    "eval/epoch/MeanPriorPixelAccuracy",
    "eval/epoch/MeanPriorForegroundBinaryMIOU"
]

col_mapping = {
    "eval/epoch/MeanForegroundBinaryMIOU": "mIoU",
    "eval/epoch/MeanPixelAccuracy": "Acc.",
    "eval/epoch/MeanPriorPixelAccuracy" : "Convex Acc.",
    "eval/epoch/MeanPriorForegroundBinaryMIOU": "Convex mIoU"
}

df = comparison.metric_table(metrics, 
                             ref="last", 
                             mode="max",
                        formatting=False)

df = df.reset_index()

def extract_ft_row(row: pd.Series) -> Tuple[str, bool, str, int]:
    name = row['index']
    model = [m for m in models if m.name == name][0]
    res = extract_features(model)
    return (res['model_name'], res['joint'], res['feature_type'], res['seed'])


df[['model_name', 'joint', 'feature_type', 'seed']] = df.apply(extract_ft_row, axis=1, result_type="expand")

grouped = df.groupby(['model_name', 'joint', 'feature_type'])
display(df)

In [ ]:
# df.loc[group.index, col] = f"{mean:.3f} ± {std:.3f}"
grouped_items = []

mean_std_cols = []
for keys, group in grouped:
    # Mean and std over metrics
    row = dict()
    row['model_name'] = group['model_name'].iloc[0]
    row['joint'] = group['joint'].iloc[0]
    row['feature_type'] = group['feature_type'].iloc[0]
    row['seeds'] = ", ".join([str(x) for x in group['seed'].sort_values().to_list()])
    row['model_index'] = ", ".join([str(x) for x in group.index.to_list()])
    for col in metrics:
        mean_std_col = ('mean ' + col, 'std ' + col, col)
        mean_std_cols.append(mean_std_col)
        row[mean_std_col[0]] = group[col].mean()
        row[mean_std_col[1]] = group[col].std()
    grouped_items.append(row)
grouped_df = pd.DataFrame(grouped_items)


# ORder
grouped_df['feature_type'] = pd.Categorical(grouped_df['feature_type'], ["xy", "feat", "featxy"])
grouped_df = grouped_df.sort_values(by=['joint', 'model_name', 'feature_type'], ascending=[True, True, True])
grouped_df

In [ ]:
# Combine columns
grouped_df_mod = grouped_df.copy()
new_named_cols = []

for col_tup in mean_std_cols:
    orig_col = col_tup[2]
    col_name = col_mapping[orig_col]
    grouped_df_mod[col_name] = grouped_df_mod[col_tup[0]].map(lambda x: f"{x:.3f}") + " ± " + grouped_df_mod[col_tup[1]].map(lambda x: f"{x:.3f}")
    if col_name not in new_named_cols:
        new_named_cols.append(col_name)

order = ["mIoU", "Convex mIoU", "Acc.", "Convex Acc."]
display_df = grouped_df_mod[['model_name', 'feature_type', 'joint'] + order]

display_df.loc[:, 'feature_type'] = display_df['feature_type'].replace({"xy": "spatial", "feat": "semantic", "featxy": "spatial + semantic"})
display_df.loc[:, 'joint'] = display_df['joint'].replace({True: "x", False: "-"})
display_df = display_df.rename(columns={"model_name": "Model", "feature_type": "Add. Input"})

display_df

In [ ]:
grouped_df

In [ ]:
print(display_df.reset_index(drop=True).to_markdown())

In [ ]:
new_named_cols# Group by joined


joint_format_df = display_df.copy()

individually_training = joint_format_df[joint_format_df['joint'] == "-"][[x for x in joint_format_df.columns if x != "joint"]]
joint_trained = joint_format_df[joint_format_df['joint'] == "x"][[x for x in joint_format_df.columns if x != "joint"]]

col_concatted = individually_training.join(joint_trained.set_index(["Model", "Add. Input"]), on=["Model", "Add. Input"], rsuffix=" joint")
col_concatted.transpose()

In [ ]:
test_df = joint_format_df.set_index(['joint', 'Model', 'Add. Input'])
test_df


In [ ]:
test_df = joint_format_df.copy(deep=True) 

test_df = test_df.rename(columns={
                        "Convex mIoU": "mIoU Convex",
                        "mIoU": "mIoU Segmentation",
                        "Convex Acc.": "Acc. Convex",
                        "Acc.": "Acc. Segmentation"
                        })
test_df = test_df.reset_index(drop=True)
test_df.index.name= "index"
test_df['id'] = test_df.index
#test_df = test_df.reset_index()

display(test_df)
wide_res_df = pd.wide_to_long(test_df, stubnames=['mIoU', "Acc."], i='id', j='kind', sep=" ", suffix='.*').reset_index()
wide_res_df['kind'] = wide_res_df['kind'].replace({"Segmentation": "Predicted Seg.", "Convex": "Convex Seg."})
wide_res_df['joint'] = wide_res_df['joint'].replace({"-": "Individually Trained", "x": "Jointly Trained"})
wide_res_df['Model'] = wide_res_df['Model'].replace({"FCNet": "FC Net", "CNNet": "CNN"})
wide_res_df['Add. Input'] = wide_res_df['Add. Input'].replace({"spatial + semantic": "spatial and semantic"})


wide_res_df = wide_res_df.set_index(['joint', 'kind', 'Model', 'Add. Input']).sort_values(['joint', 'kind', 'Model', 'Add. Input'], ascending=[True, False, True, True])
wide_res_df = wide_res_df.drop(columns=['id'])

wide_res_df = wide_res_df.rename(columns={ "mIoU": "mIoU ↑", "Acc.": "Acc. ↑"})
wide_res_df

In [ ]:
print(latex_postprocessor(wide_res_df.style.to_latex(label=f"tab:results_eval", hrules=True, caption=f"Evaluation of joint and individual trained networks.", position_float='centering')))

In [ ]:
# Group for Add_input
texts = []
for key, group in wide_res_df.groupby(['Model', 'Add. Input']):
    print(key)
    #group = group.copy().drop(columns=['Add. Input'])
    group = group.reset_index().set_index(['joint', 'kind'])
    group = group.drop(columns=['Add. Input', 'Model'])
    group = group.transpose()
    group.columns.names = ["", ""]
    display(group)
    texts.append(group.style.to_latex(label=f"tab:eval_{key[0]}_{key[1].replace(' and ', '_')}", hrules=True, multicol_align="c", caption=f"Evaluation of {key[0]} with {key[1]} features as additional input.", position_float='centering'))
for latex in texts:
    print(latex)

In [ ]:
grouped_df

In [ ]:
plot_df = grouped_df.copy()

plot_df.loc[:, 'feature_type'] = plot_df['feature_type'].replace({"xy": "spatial", "feat": "semantic", "featxy": "spatial + semantic"})
plot_df.loc[:, 'joint'] = plot_df['joint'].replace({True: "Joint", False: "Sequential"})
plot_df.loc[:, 'model_name'] = plot_df['model_name'].replace({"CNNet": "CNN", "FCNet": "FCN"})
plot_df = plot_df.rename(columns={"mean eval/epoch/MeanForegroundBinaryMIOU": 
                                  "Seg. mIoU", "mean eval/epoch/MeanPriorForegroundBinaryMIOU": "Convex mIoU",
                                  "std eval/epoch/MeanForegroundBinaryMIOU": "Seg. mIoU err",
                                  "std eval/epoch/MeanPriorForegroundBinaryMIOU": "Convex mIoU err",
                                  })
plot_df = plot_df[['model_name', 'joint', 'feature_type', 'Seg. mIoU', 'Convex mIoU', 'Seg. mIoU err', 'Convex mIoU err']]

plot_df 

In [ ]:
from typing import Optional
from matplotlib import pyplot as plt
from matplotlib.axes import Axes
from matplotlib.image import AxesImage
from awesome.run.functions import saveable
from itertools import chain, product
import numpy as np
from matplotlib.colors import to_rgba, to_hex
import matplotlib.patches as mpatches
nets = ["FCN", "CNN"]
feat = ["semantic", "spatial + semantic"]
method = ["Sequential", "Joint"]
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica"
})
@saveable()
def plot_asbar(size: float = 5):
    fig, axes = plt.subplots(1, 1, figsize=(size, size * 0.5),sharey=True)


    cmap = plt.get_cmap("tab10")
    c1 = cmap(0)
    c2 = cmap(1)
    plt.rcParams["hatch.linewidth"] = 4
    hatch_alpha = 0.7
    c1_h = to_rgba(c1, alpha=hatch_alpha)
    c1_h = to_hex(c1_h, keep_alpha=True)

    c1_h = "#54b0f0"
    c2_h = to_rgba(c2, alpha=hatch_alpha)
    c2_h = to_hex(c2_h, keep_alpha=True)
    c2_h = "#faa85f"

    centers = []
    xlabels = []
    for i, k in enumerate(product(nets, feat)):
        ax: Axes = axes

        network = k[0]
        _feat = k[1]
        df = plot_df[(plot_df['model_name'] == network) & (plot_df['feature_type'] == _feat)]
        seq_df = df[df['joint'] == "Sequential"]
        joint_df = df[df['joint'] == "Joint"]
        start = i * 5
        center = start + 1.5
        centers.append(center)
        if "+" in k[1]:
            vals = k[1].split("+")
            vals = [x.strip() for x in vals]
            vals[-1] = "+ "+ vals[-1]
            k = (k[0],) + tuple(vals)
        xlabels.append(k)
        ax.bar(x=start + np.array(list(range(4))), 
            height=[
                seq_df['Seg. mIoU'].item(), seq_df['Convex mIoU'].item(),
                joint_df['Seg. mIoU'].item(), joint_df['Convex mIoU'].item()
                ],
                color = [c1, c2, c1, c2],
                edgecolor=[c1_h, c2_h, c1_h, c2_h],
                hatch = ["", "","//", "//"],
        )
        x = start + np.array(list(range(4)))
        ax.errorbar(x=x, 
            y=[
                seq_df['Seg. mIoU'].item(), seq_df['Convex mIoU'].item(),
                joint_df['Seg. mIoU'].item(), joint_df['Convex mIoU'].item()
                ],
                fmt="", linestyle='', color="gray",capthick=3,
                yerr=[
                    seq_df['Seg. mIoU err'].item(), seq_df['Convex mIoU err'].item(),
                    joint_df['Seg. mIoU err'].item(), joint_df['Convex mIoU err'].item()
                ]
        )
    ax.set_xticks(centers, ["\n".join(x) for x in xlabels])

    label_colors = {
        "Seg.": c1,
        "Convex Seg.": c2,
    }
    patches = [mpatches.Patch(color=v, label=k) for k, v in  label_colors.items()]

    patches.append(mpatches.Patch(facecolor=c1, label="Joint Seg.", hatch="//", edgecolor=c1_h))
    patches.append(mpatches.Patch(facecolor=c2, label="Joint Convex Seg.", hatch="//", edgecolor=c2_h))

    plt.legend(handles=patches, loc="lower right")

    ax.set_ylabel("IoU")
    ax.grid(axis="y", linestyle='-', linewidth=1)
    return fig
fig = plot_asbar(save=True, path="model_input_results", tight_layout=True,ext=["png", "pdf"], override=True)
order = ['Seg.', 'Convex Proj.', 'Joint Seg.', 'Joint Proj.']
fig

In [ ]:
comparison.open_folder()

In [ ]:
ax.set_xticks(centers, ["\n".join(x) for x in xlabels])
fig

In [ ]:
fig

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# example data
x = np.arange(0.1, 4, 0.5)
y = np.exp(-x)

# example error bar values that vary with x-position
error = 0.1 + 0.2 * x

fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True)
ax0.errorbar(x, y, yerr=error, fmt='-o')
ax0.set_title('variable, symmetric error')

# error bar values w/ different -/+ errors that
# also vary with the x-position
# lower_error = 0.4 * error
# upper_error = error
# asymmetric_error = [lower_error, upper_error]

# ax1.errorbar(x, y, xerr=asymmetric_error, fmt='o')
# ax1.set_title('variable, asymmetric error')
# ax1.set_yscale('log')
plt.show()

In [ ]:
error